In [1]:
import numpy as np
import matplotlib.pyplot as plt
import Circuit
from EC07 import EC07
import pandas as pd
from numpy import random

In [2]:
frequencies = Circuit.dec_step(1e-2, 1e5, 72)

In [3]:
import random
# from openpyxl import Workbook
samples=50000
# Generate random values for three variables
Rs = [random.uniform(15,35) for _ in range(samples)]
R1 = [random.uniform(15,35) for _ in range(samples)]
R2 = [random.uniform(15,35) for _ in range(samples)]
R3 = [random.uniform(15,35) for _ in range(samples)]
CPE_Y1 = [random.uniform(1e-6, 1e-3) for _ in range(samples)]
CPE_n1 = [random.uniform(0.5, 1) for _ in range(samples)]
CPE_Y2 = [random.uniform(1e-6, 1e-3) for _ in range(samples)]
CPE_n2 = [random.uniform(0.5, 1) for _ in range(samples)]
CPE_Y3 = [random.uniform(1e-6, 1e-3) for _ in range(samples)]
CPE_n3 = [random.uniform(0.5, 1) for _ in range(samples)]

In [4]:
output_array = np.stack((Rs,R1,R2,R3, CPE_Y1, CPE_n1,CPE_Y2, CPE_n2,CPE_Y3, CPE_n3), axis=1)

In [5]:
num_rows = len(output_array)
num_rows

50000

In [6]:
num_freq = len(output_array)
num_rows = 72
num_columns = 3

# Create an empty numpy array to store the frequency response data
frequency_response_array_7 = np.empty((num_freq, num_rows, num_columns))
# len(frequency_response_array[0][0])

In [7]:
# As output_array is a 2D NumPy array
for idx in range(output_array.shape[0]):
    row = output_array[idx]
    b = EC07(*row)
    Z = b.freq_response(frequencies)

    # Store the frequency values  in the first column
    frequency_response_array_7[idx, :, 0] = frequencies

    # Store the Absolute Value of Z in the second column
    frequency_response_array_7[idx, :, 1] = np.abs(Z)

    # Store the Phase values of Z in the third column
    frequency_response_array_7[idx, :, 2] = np.angle(Z)


# Now, frequency_response_array has the shape (50000, 72, 3)



In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

# Assuming frequency_response_array has shape (50000, 72, 3) and output_array has shape (50000, 3)

# Reshape frequency_response_array to have shape (50000, 216)
X = frequency_response_array_7.reshape((frequency_response_array_7.shape[0], -1))

y = output_array

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Initialize and train the linear regression model
# poly=PolynomialFeatures(degree=3)
# X_train1=poly.fit_transform(X_train)
# X_test1=poly.transform(X_test)
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate the model
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print("Training R^2 Score:", train_score)
print("Testing R^2 Score:", test_score)


Training R^2 Score: 0.37684538982047144
Testing R^2 Score: 0.3683440477565106


In [11]:
for i in range(1000):    
    random_idx = np.random.randint(0, len(frequency_response_array_7))
    # Select the random element from frequency_response_array
    random_element = frequency_response_array_7[random_idx]

    # Reshape the random element to match the input shape expected by the model
    reshaped_element = random_element.reshape(1, -1)

    # Predict the corresponding output value
    predicted_output = model.predict(reshaped_element)

    # Extract the corresponding actual output value from the output array
    actual_output = output_array[random_idx]

    # Print the predicted and actual output values for comparison
    print("Predicted Output:",predicted_output)
    print("Actual Output:",actual_output)


Predicted Output: [[2.86220703e+01 3.00937500e+01 2.86875000e+01 3.07187500e+01
  2.05993652e-04 7.74414062e-01 1.59263611e-04 7.71484375e-01
  2.28881836e-04 7.89062500e-01]]
Actual Output: [2.82928118e+01 3.42659453e+01 3.21071386e+01 2.34905802e+01
 5.21881995e-06 9.56483710e-01 2.96981146e-04 6.24193184e-01
 3.42886281e-04 9.39320261e-01]
Predicted Output: [[1.31513672e+01 2.42812500e+01 2.46875000e+01 2.49062500e+01
  4.95910645e-04 7.19726562e-01 4.86373901e-04 7.17285156e-01
  4.15802002e-04 7.00195312e-01]]
Actual Output: [1.55317550e+01 1.87519860e+01 3.36274722e+01 1.91419719e+01
 3.82639131e-04 9.20391682e-01 6.82116702e-04 6.92695266e-01
 1.16486573e-05 5.70641473e-01]
Predicted Output: [[2.81865234e+01 2.57187500e+01 2.57500000e+01 2.60937500e+01
  4.69207764e-04 8.25195312e-01 4.69207764e-04 8.34960938e-01
  4.61578369e-04 8.33007812e-01]]
Actual Output: [2.81307051e+01 2.70787574e+01 2.70235141e+01 2.36042453e+01
 8.78600411e-04 7.95639254e-01 3.02536133e-04 9.52211888e-

In [12]:
from sklearn.metrics import mean_squared_error,accuracy_score

# Predict outputs for the entire test set
predicted_outputs = model.predict(X_test)

# Compute mean squared error
mse = mean_squared_error(y_test, predicted_outputs)
accuracy=100-mse
# Print the mean squared error
print("Mean Squared Error:", mse)
print("Accuracy:",accuracy)


Mean Squared Error: 6.784362942005518
Accuracy: 93.21563705799448


In [13]:
np.save('frequency_response_array_7.npy',frequency_response_array_7)